# Scripts que devuelve las diferencias entre los ficheros

Importamos

In [1]:
import json
import os
import pandas as pd
from difflib import SequenceMatcher
import jellyfish
from geopy.geocoders import Nominatim
geolocator = Nominatim()
from geopy.distance import great_circle
import folium
from folium.plugins import MarkerCluster

In [2]:
#Center coordinates
SF_COORDINATES = (40.4168, -3.7038)

# for speed purposes
MAX_RECORDS = 10

In [3]:
def getDistance_meters(x,y):
    try:
        return great_circle(x, y).kilometers*1000.0
    except:
        return 0

In [41]:
getDistance_meters([40.3525299,-3.7040253],[40.3525234,-3.7036531])

31.556980047597786

In [4]:
os.chdir('/Volumes/MacintoshHD/_GitHub/doctordata/api/data/')

Creamos un data para los datos de OSM

In [5]:
with open('drinking_water.geojson') as f:
    OSM = json.load(f)
f.close()

In [6]:
OSM['features']

[{'geometry': {'coordinates': [-3.6957228, 40.413707], 'type': 'Point'},
  'id': 'node/167302670',
  'properties': {'@id': 'node/167302670', 'amenity': 'drinking_water'},
  'type': 'Feature'},
 {'geometry': {'coordinates': [-3.6761993, 40.4458451], 'type': 'Point'},
  'id': 'node/297655024',
  'properties': {'@id': 'node/297655024', 'amenity': 'drinking_water'},
  'type': 'Feature'},
 {'geometry': {'coordinates': [-3.6839934, 40.4672427], 'type': 'Point'},
  'id': 'node/297655027',
  'properties': {'@id': 'node/297655027', 'amenity': 'drinking_water'},
  'type': 'Feature'},
 {'geometry': {'coordinates': [-3.6630123, 40.4232182], 'type': 'Point'},
  'id': 'node/297682006',
  'properties': {'@id': 'node/297682006',
   'amenity': 'drinking_water',
   'name': 'Fuente del Berro'},
  'type': 'Feature'},
 {'geometry': {'coordinates': [-3.669598, 40.4812719], 'type': 'Point'},
  'id': 'node/307782191',
  'properties': {'@id': 'node/307782191',
   'amenity': 'drinking_water',
   'name': 'Fuente

In [7]:
ids = [OSM['features'][i]['id'] for i in range(len(OSM['features']))]
lat = [OSM['features'][i]['geometry']['coordinates'][1] for i in range(len(OSM['features']))]
lon = [OSM['features'][i]['geometry']['coordinates'][0] for i in range(len(OSM['features']))]
ayu = ids[:]

In [8]:
dataOSM = pd.DataFrame([ids,lat,lon])

In [9]:
dataOSM = dataOSM.transpose()
dataOSM.columns = ['ids','lat','lon']
dataOSM.lat = dataOSM.lat.astype('float32')
dataOSM.lon = dataOSM.lon.astype('float32')
dataOSM.head(2)

,ids,lat,lon
0,node/167302670,40.413708,-3.695723
1,node/297655024,40.445847,-3.676199


In [10]:
dataOSM['coord'] = list(zip(dataOSM.lat, dataOSM.lon))

In [11]:
dataOSM = dataOSM.sort_values('lat')

In [18]:
AYU = pd.read_csv('InventarioFuentes.csv', encoding='latin-1', delimiter=';')
AYU = AYU[AYU.LATITUD != 'Error']
AYU = AYU[AYU.LONGITUD != 'Error']
AYU.LATITUD = AYU.LATITUD.astype('float32')
AYU.LONGITUD = AYU.LONGITUD.astype('float32')
AYU = AYU.sort_values('LATITUD')
AYU['id_OSM'] = pd.Series()
AYU['coord_OSM'] = pd.Series()
AYU.head(2)

,CODIGO,ZONA VERDE / VIA PUBLICA,DISTRITO,DIRECCION,COMPLEMENTO DE DIRECCIÓN,COORD. X,COORD. Y,SISTEMA DE COORDENADAS,ESTADO,OBSERVACIONES,FECHA DATOS,LONGITUD,LATITUD,id_OSM,coord_OSM
1337,FUE_06-17_0000000113,VP,VILLAVERDE,AV. DE LOS ROSALES 407,NaN,"443914,6707","4464487,999",ETRS89 / UTM Zona 30N,EN SERVICIO,NaN,3-nov.-17,-3.660224,40.329044,NaN,NaN
1285,FUE_06-17_0000000060,ZV,VILLAVERDE,INST. DEP. COL. MARCONI,NaN,"440662,8099","4465018,166",ETRS89 / UTM Zona 30N,EN SERVICIO,NaN,3-nov.-17,-3.698551,40.333595,NaN,NaN


In [19]:
AYU['coord'] = list(zip(AYU.LATITUD, AYU.LONGITUD))

In [20]:
AYU = AYU.reset_index()

In [21]:
dataOSM.coord.iloc[0]

(40.329044, -3.6602237)

In [22]:
getDistance_meters(dataOSM.coord.iloc[1],AYU.coord.iloc[1])

0.08085875116369658

In [23]:
AYU.head(2)

,index,CODIGO,ZONA VERDE / VIA PUBLICA,DISTRITO,DIRECCION,COMPLEMENTO DE DIRECCIÓN,COORD. X,COORD. Y,SISTEMA DE COORDENADAS,ESTADO,OBSERVACIONES,FECHA DATOS,LONGITUD,LATITUD,id_OSM,coord_OSM,coord
0,1337,FUE_06-17_0000000113,VP,VILLAVERDE,AV. DE LOS ROSALES 407,NaN,"443914,6707","4464487,999",ETRS89 / UTM Zona 30N,EN SERVICIO,NaN,3-nov.-17,-3.660224,40.329044,NaN,NaN,"(40.329, -3.66022)"
1,1285,FUE_06-17_0000000060,ZV,VILLAVERDE,INST. DEP. COL. MARCONI,NaN,"440662,8099","4465018,166",ETRS89 / UTM Zona 30N,EN SERVICIO,NaN,3-nov.-17,-3.698551,40.333595,NaN,NaN,"(40.3336, -3.69855)"


In [26]:
def get_id(x):
    for i in range(len(ids)):
        point = [lat[i],lon[i]]
        if getDistance_meters(x,point) <5:
            ayu[i] = x
            return ids[i]
    return 'No match'

def get_close(x):
    for i in range(len(ids)):
        point = [lat[i],lon[i]]
        if getDistance_meters(x,point) <30:
            ayu[i] = x
            return ids[i]
    return 'No match'

In [28]:
AYU.id_OSM = AYU.coord.apply(lambda x: get_id(x)) 
AYU_NO = AYU[AYU.id_OSM == 'No match']
AYU_OK = AYU[AYU.id_OSM != 'No match']

In [29]:
AYU_CLOSE = AYU_NO.copy()
AYU_CLOSE.id_OSM = AYU_NO.coord.apply(lambda x: get_close(x))
AYU_NO = AYU_CLOSE[AYU_CLOSE.id_OSM == 'No match']
AYU_CLOSE = AYU_CLOSE[AYU_CLOSE.id_OSM != 'No match']

## Close to edit

In [30]:
map = folium.Map(location=SF_COORDINATES, zoom_start=11)
marker_cluster = MarkerCluster().add_to(map)
for i in range(len(AYU_CLOSE.id_OSM.values)):
    folium.Marker(icon=folium.Icon(color='blue'),popup=folium.Popup(AYU_CLOSE.id_OSM.values[i], parse_html=True),
        location = (AYU_CLOSE.LATITUD.values[i],AYU_CLOSE.LONGITUD.values[i])).add_to(marker_cluster)
map

## Missing in OSM

In [31]:
map = folium.Map(location=SF_COORDINATES, zoom_start=11)
marker_cluster = MarkerCluster().add_to(map)
for i in range(len(AYU_NO.id_OSM.values[:100])):
    folium.Marker(icon=folium.Icon(color='blue'),popup=folium.Popup(AYU_NO.id_OSM.values[i], parse_html=True),
        location = (AYU_NO.LATITUD.values[i],AYU_NO.LONGITUD.values[i])).add_to(marker_cluster)
map

## Missing in Ayu website

In [32]:
AYU_MISSING = []
for i in range(len(ids)):
    if isinstance(ayu[i], str):
        AYU_MISSING.append((ids[i],lat[i],lon[i]))

In [33]:
AYU_MISSING = pd.DataFrame(AYU_MISSING)
AYU_MISSING.columns = ['id_OSM','LATITUD','LONGITUD']

In [34]:
map = folium.Map(location=SF_COORDINATES, zoom_start=11)
marker_cluster = MarkerCluster().add_to(map)
for i in range(len(AYU_MISSING.id_OSM.values)):
    folium.Marker(icon=folium.Icon(color='green'),popup=folium.Popup(AYU_MISSING.id_OSM.values[i], parse_html=True),
        location = (AYU_MISSING.LATITUD.values[i],AYU_MISSING.LONGITUD.values[i])).add_to(marker_cluster)
map

In [35]:
AYU_MISSING = AYU_MISSING.set_index('id_OSM')

In [36]:
mydic = AYU_MISSING.to_dict(orient='index')

In [37]:
mydic.keys()

dict_keys(['node/1504407832', 'node/1048614568', 'node/4065603214', 'node/4396225695', 'node/4333068049', 'node/4331338099', 'node/1440939173', 'node/4330503376', 'node/1440939172', 'node/2946271316', 'node/1743699840', 'node/1440939175', 'node/1052390687', 'node/1436712916', 'node/1892827827', 'node/3697360448', 'node/4470353161', 'node/991793741', 'node/4265348136', 'node/4427657680', 'node/4907732166', 'node/1435923179', 'node/4332033579', 'node/5017117015', 'node/3150808038', 'node/4331338109', 'node/559824976', 'node/4332033577', 'node/857157901', 'node/4332033581', 'node/4062129993', 'node/3697360449', 'node/3697360452', 'node/3577606786', 'node/2624160636', 'node/4413513647', 'node/2623339508', 'node/4340080832', 'node/4219016930', 'node/4347754197', 'node/3951792357', 'node/1228619939', 'node/4332033578', 'node/4333068048', 'node/1517405751', 'node/3500619801', 'node/4332132438', 'node/1992836768', 'node/3510203609', 'node/1325487024', 'node/4376883228', 'node/4330516734', 'nod

In [38]:
import json
print(json.dumps(mydic, indent=4))

{
    "node/1504407832": {
        "LONGITUD": -3.5909,
        "LATITUD": 40.4874
    },
    "node/1048614568": {
        "LONGITUD": -3.6010425,
        "LATITUD": 40.4221651
    },
    "node/4065603214": {
        "LONGITUD": -3.6579993,
        "LATITUD": 40.4511638
    },
    "node/4396225695": {
        "LONGITUD": -3.6404463,
        "LATITUD": 40.4705536
    },
    "node/4333068049": {
        "LONGITUD": -3.7246408,
        "LATITUD": 40.4355529
    },
    "node/4331338099": {
        "LONGITUD": -3.7186942,
        "LATITUD": 40.4252315
    },
    "node/1440939173": {
        "LONGITUD": -3.7515119,
        "LATITUD": 40.4113367
    },
    "node/4330503376": {
        "LONGITUD": -3.700859,
        "LATITUD": 40.3548719
    },
    "node/1440939172": {
        "LONGITUD": -3.7507684,
        "LATITUD": 40.4098531
    },
    "node/2946271316": {
        "LONGITUD": -3.720387,
        "LATITUD": 40.4735705
    },
    "node/1743699840": {
        "LONGITUD": -3.6935925,
        "